<a href="https://colab.research.google.com/github/Xuehui-Jiang/Uni-Mannheim-Master-Thesis/blob/main/Experimental_Evaluation/Experiment_Forbes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cloning the pyRDF2Vec repo
!git clone https://github.com/IBCNServices/pyRDF2Vec.git

Cloning into 'pyRDF2Vec'...
remote: Enumerating objects: 7412, done.
remote: Counting objects: 100% (380/380), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 7412 (delta 234), reused 259 (delta 133), pack-reused 7032
Receiving objects: 100% (7412/7412), 5.32 MiB | 12.35 MiB/s, done.
Resolving deltas: 100% (5038/5038), done.


In [ ]:
!pip uninstall community
!pip install python-louvain
!pip install pyRDF2Vec --upgrade

Found existing installation: community 1.0.0b1
Uninstalling community-1.0.0b1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/community-1.0.0b1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/community/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/community/community_louvain.py
    /usr/local/lib/python3.10/dist-packages/community/community_status.py
Proceed (Y/n)? Y
  Successfully uninstalled community-1.0.0b1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install rdflib
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00


In [ ]:
import random
import warnings
warnings.filterwarnings('ignore')
import functools
import itertools
from typing import List, Sequence, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rdflib
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC

import sys
sys.path.append('pyRDF2Vec')
sys.path.append('pyRDF2Vec/pyrdf2vec')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.walkers import RandomWalker

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


Saving completedataset.tsv to completedataset.tsv
User uploaded file "completedataset.tsv" with length 185776 bytes


In [ ]:
# load training and testing entities
complete_df = pd.read_csv('completedataset.tsv', sep='\t')

entities = complete_df['DBpedia_URI'].tolist()
labels = complete_df['label'].tolist()
reg_targets = complete_df['Market_Value'].tolist()

In [ ]:
# Filter out entities and relationships directly related to Forbes Dataset
with open("/content/drive/My Drive/dbpedia_files/direct_merged_dbpedia.ttl", "r") as file, open("direct_Forbes_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2:
            # Check if the subject or object of the triple is in our forbes URL list
            if triple[0].strip('<>') in entities or triple[2].strip('<>') in entities:
                out_file.write(line)

In [ ]:
# Filter out entities and relationships directly or indirectly related to Cities Dataset
entities_related_to_cities = set(entities)
new_entities = set(entities)
iteration_count = 0

while new_entities and iteration_count < 2:
    temp_entities = set()
    with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file:
        for line in file:
            triple = line.strip().split()
            if len(triple) > 2 and (triple[0].strip('<>') in new_entities or triple[2].strip('<>') in new_entities):
                    temp_entities.add(triple[0].strip('<>'))
                    temp_entities.add(triple[2].strip('<>'))

    new_entities = temp_entities - entities_related_to_cities
    entities_related_to_cities.update(new_entities)
    iteration_count += 1

# Finally, we can write all triples that are directly or indirectly related to entities in the `entities_related_to_cities` collection
with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file, open("indirect_Forbes_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2 and (triple[0].strip('<>') in entities_related_to_cities or triple[2].strip('<>') in entities_related_to_cities):
            out_file.write(line)


In [ ]:
# Define the list of files
files_to_merge = [
    "direct_Forbes_filtered_dbpedia.ttl",
    "indirect_Forbes_filtered_dbpedia.ttl"
]

# Merge files
with open("Forbes_filtered_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open(file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


In [ ]:
input_file = "Forbes_filtered_dbpedia.ttl"
output_file = "cleaned_Forbes_filtered_dbpedia.ttl"

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Determine whether it is a comment or a blank line
        if line.startswith("#") or line.strip() == "":
            continue

        # Split triples
        triple = line.strip().split()

        # Make sure this is a complete triplet
        if len(triple) > 2:
            # Check whether the object starts with "http://dbpedia.org/resource/"
            if triple[2].startswith("<http://dbpedia.org/resource/"):
                outfile.write(line)

print(f"Finished filtering {input_file} and saved the result to {output_file}")


Finished filtering Forbes_filtered_dbpedia.ttl and saved the result to cleaned_Forbes_filtered_dbpedia.ttl


In [ ]:
!head -n 10 cleaned_Forbes_filtered_dbpedia.ttl

<http://dbpedia.org/resource/AOL> <http://dbpedia.org/property/founders> <http://dbpedia.org/resource/Marc_Seriff> .
<http://dbpedia.org/resource/AOL> <http://dbpedia.org/property/founders> <http://dbpedia.org/resource/Steve_Case> .
<http://dbpedia.org/resource/AOL> <http://dbpedia.org/property/founders> <http://dbpedia.org/resource/Jim_Kimsey> .
<http://dbpedia.org/resource/Alan_Kay> <http://dbpedia.org/property/workplaces> <http://dbpedia.org/resource/Hewlett-Packard> .
<http://dbpedia.org/resource/Arkansas> <http://dbpedia.org/property/flower> <http://dbpedia.org/resource/Apple> .
<http://dbpedia.org/resource/Adobe_Systems> <http://dbpedia.org/property/tradedAs> <http://dbpedia.org/resource/NASDAQ-100> .
<http://dbpedia.org/resource/Adobe_Systems> <http://dbpedia.org/property/tradedAs> <http://dbpedia.org/resource/S&P_500> .
<http://dbpedia.org/resource/Accelerated_Graphics_Port> <http://dbpedia.org/property/inventName> <http://dbpedia.org/resource/Intel> .
<http://dbpedia.org/resou

In [ ]:
import shutil

# 定义要保存的文件路径
destination_path = "/content/drive/My Drive/dbpedia_files"

# 复制文件到Google Drive
shutil.copy("cleaned_Forbes_filtered_dbpedia.ttl", destination_path)

print(f"File saved to {destination_path}")

File saved to /content/drive/My Drive/dbpedia_files


In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("/content/drive/My Drive/dbpedia_files/cleaned_Forbes_filtered_dbpedia.ttl", fmt='turtle')

kgentities = kg._entities

In [ ]:
for e in entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

http://dbpedia.org/resource/HSBC_Holdings
http://dbpedia.org/resource/Mitsubishi_UFJ_Financial
http://dbpedia.org/resource/Westpac_Banking_Group
http://dbpedia.org/resource/Goldman_Sachs_Group
http://dbpedia.org/resource/Ford_Motor
http://dbpedia.org/resource/Ping_An_Insurance_Group
http://dbpedia.org/resource/Nissan_Motor
http://dbpedia.org/resource/Hyundai_Motor
http://dbpedia.org/resource/Saudi_Basic_Industries
http://dbpedia.org/resource/China_Life_Insurance
http://dbpedia.org/resource/China_Minsheng_Banking
http://dbpedia.org/resource/Mitsubishi_Corp
http://dbpedia.org/resource/Cnooc
http://dbpedia.org/resource/Hon_Hai_Precision
http://dbpedia.org/resource/China_Shenhua_Energy
http://dbpedia.org/resource/US_Bancorp
http://dbpedia.org/resource/Nordea_Bank
http://dbpedia.org/resource/China_Citic_Bank
http://dbpedia.org/resource/Credit_Suisse_Group
http://dbpedia.org/resource/Industrial_Bank
http://dbpedia.org/resource/PTT_PCL
http://dbpedia.org/resource/UniCredit_Group
http://dbpedi

In [ ]:
# Entities not in KG
absent_entities = [
    "http://dbpedia.org/resource/HSBC_Holdings",
    "http://dbpedia.org/resource/Mitsubishi_UFJ_Financial",
    "http://dbpedia.org/resource/Westpac_Banking_Group",
    "http://dbpedia.org/resource/Goldman_Sachs_Group",
    "http://dbpedia.org/resource/Ford_Motor",
    "http://dbpedia.org/resource/Ping_An_Insurance_Group",
    "http://dbpedia.org/resource/Nissan_Motor",
    "http://dbpedia.org/resource/Hyundai_Motor",
    "http://dbpedia.org/resource/Saudi_Basic_Industries",
    "http://dbpedia.org/resource/China_Life_Insurance",
    "http://dbpedia.org/resource/China_Minsheng_Banking",
    "http://dbpedia.org/resource/Mitsubishi_Corp",
    "http://dbpedia.org/resource/Cnooc",
    "http://dbpedia.org/resource/Hon_Hai_Precision",
    "http://dbpedia.org/resource/China_Shenhua_Energy",
    "http://dbpedia.org/resource/US_Bancorp",
    "http://dbpedia.org/resource/Nordea_Bank",
    "http://dbpedia.org/resource/China_Citic_Bank",
    "http://dbpedia.org/resource/Credit_Suisse_Group",
    "http://dbpedia.org/resource/Industrial_Bank",
    "http://dbpedia.org/resource/PTT_PCL",
    "http://dbpedia.org/resource/UniCredit_Group",
    "http://dbpedia.org/resource/Glencore_International",
    "http://dbpedia.org/resource/TNK-BP_Holding",
    "http://dbpedia.org/resource/Canadian_Imperial_Bank",
    "http://dbpedia.org/resource/Bank_of_New_York_Mellon",
    "http://dbpedia.org/resource/Legal_&_General_Group",
    "http://dbpedia.org/resource/Costco_Wholesale",
    "http://dbpedia.org/resource/ACE",
    "http://dbpedia.org/resource/East_Japan_Railway",
    "http://dbpedia.org/resource/Seven_&_I_Holdings",
    "http://dbpedia.org/resource/People's_Insurance_Company",
    "http://dbpedia.org/resource/Taiwan_Semiconductor",
    "http://dbpedia.org/resource/BCE",
    "http://dbpedia.org/resource/SEB",
    "http://dbpedia.org/resource/Svenska_Handelsbanken",
    "http://dbpedia.org/resource/Shinhan_Financial_Group",
    "http://dbpedia.org/resource/Woolworths",
    "http://dbpedia.org/resource/Heineken_Holding",
    "http://dbpedia.org/resource/Power_Corp_of_Canada",
    "http://dbpedia.org/resource/OMV_Group",
    "http://dbpedia.org/resource/China_Railway_Construction",
    "http://dbpedia.org/resource/China_Railway_Group",
    "http://dbpedia.org/resource/KBC_Group",
    "http://dbpedia.org/resource/Indian_Oil",
    "http://dbpedia.org/resource/HCA_Holdings",
    "http://dbpedia.org/resource/Saudi_Telecom",
    "http://dbpedia.org/resource/Baoshan_Iron_&_Steel",
    "http://dbpedia.org/resource/AMP",
    "http://dbpedia.org/resource/Woori_Finance_Holdings",
    "http://dbpedia.org/resource/FirstRand",
    "http://dbpedia.org/resource/PG&E",
    "http://dbpedia.org/resource/Deutsche_Lufthansa",
    "http://dbpedia.org/resource/SK_Holdings",
    "http://dbpedia.org/resource/Huaxia_Bank",
    "http://dbpedia.org/resource/QBE_Insurance_Group",
    "http://dbpedia.org/resource/Generali_Group",
    "http://dbpedia.org/resource/CLP_Holdings",
    "http://dbpedia.org/resource/Grupo_Mexico",
    "http://dbpedia.org/resource/Lincoln_National",
    "http://dbpedia.org/resource/Fifth_Third_Bancorp",
    "http://dbpedia.org/resource/Porsche_Automobil_Holding",
    "http://dbpedia.org/resource/Kirin_Holdings",
    "http://dbpedia.org/resource/Keppel_Corp",
    "http://dbpedia.org/resource/China_Coal_Energy",
    "http://dbpedia.org/resource/GFNorte",
    "http://dbpedia.org/resource/Dongfeng_Motor_Group",
    "http://dbpedia.org/resource/Tokio_Marine_Holdings",
    "http://dbpedia.org/resource/Qatar_National_Bank",
    "http://dbpedia.org/resource/J_Sainsbury",
    "http://dbpedia.org/resource/Anglo_American",
    "http://dbpedia.org/resource/CaixaBank",
    "http://dbpedia.org/resource/Korea_Electric_Power",
    "http://dbpedia.org/resource/Sumitomo_Electric",
    "http://dbpedia.org/resource/Regions_Financial",
    "http://dbpedia.org/resource/Precision_Castparts",
    "http://dbpedia.org/resource/Franklin_Resources",
    "http://dbpedia.org/resource/Swatch_Group",
    "http://dbpedia.org/resource/Yum_Brands",
    "http://dbpedia.org/resource/Hartford_Financial_Services",
    "http://dbpedia.org/resource/HJ_Heinz",
    "http://dbpedia.org/resource/Mitsubishi_Chemical",
    "http://dbpedia.org/resource/Datang_International_Power",
    "http://dbpedia.org/resource/Ensco",
    "http://dbpedia.org/resource/Evergrande_Real_Estate",
    "http://dbpedia.org/resource/Grupo_Aval",
    "http://dbpedia.org/resource/Daiwa_House_Industry",
    "http://dbpedia.org/resource/Gree_Electric_Appliances",
    "http://dbpedia.org/resource/Erste_Group_Bank",
    "http://dbpedia.org/resource/Yanzhou_Coal_Mining",
    "http://dbpedia.org/resource/Parker-Hannifin",
    "http://dbpedia.org/resource/Deutsche_Boerse",
    "http://dbpedia.org/resource/Metro_Group",
    "http://dbpedia.org/resource/Lenovo_Group",
    "http://dbpedia.org/resource/VakifBank",
    "http://dbpedia.org/resource/ALFA",
    "http://dbpedia.org/resource/KeyCorp",
    "http://dbpedia.org/resource/Anhui_Conch_Cement",
    "http://dbpedia.org/resource/Luxottica_Group",
    "http://dbpedia.org/resource/BRF-Brasil_Foods",
    "http://dbpedia.org/resource/Asahi_Glass",
    "http://dbpedia.org/resource/West_Japan_Railway",
    "http://dbpedia.org/resource/SCOR",
    "http://dbpedia.org/resource/Biogen_Idec",
    "http://dbpedia.org/resource/Kansai_Electric_Power",
    "http://dbpedia.org/resource/KGHM_Polska_Miedz",
    "http://dbpedia.org/resource/Citic_Pacific",
    "http://dbpedia.org/resource/IDGC_Holding",
    "http://dbpedia.org/resource/Citic_Securities",
    "http://dbpedia.org/resource/GD_Power_Development",
    "http://dbpedia.org/resource/Turk_Telekom",
    "http://dbpedia.org/resource/TRW_Automotive_Holdings",
    "http://dbpedia.org/resource/CF_Industries_Holdings",
    "http://dbpedia.org/resource/Tokyo_Electric_Power",
    "http://dbpedia.org/resource/Mazda_Motor",
    "http://dbpedia.org/resource/Aluminum_Corp_of_China",
    "http://dbpedia.org/resource/Saudi_British_Bank",
    "http://dbpedia.org/resource/Asustek_Computer",
    "http://dbpedia.org/resource/Zijin_Mining_Group",
    "http://dbpedia.org/resource/Suning_Appliance",
    "http://dbpedia.org/resource/Genuine_Parts",
    "http://dbpedia.org/resource/Uni-President",
    "http://dbpedia.org/resource/WW_Grainger",
    "http://dbpedia.org/resource/Campbell_Soup",
    "http://dbpedia.org/resource/Daiwa_Securities",
    "http://dbpedia.org/resource/Kweichow_Moutai",
    "http://dbpedia.org/resource/Antarchile",
    "http://dbpedia.org/resource/Jeronimo_Martins",
    "http://dbpedia.org/resource/Petronas_Chemicals",
    "http://dbpedia.org/resource/Electrolux_Group",
    "http://dbpedia.org/resource/Sinopharm_Group",
    "http://dbpedia.org/resource/Great_Wall_Motor",
    "http://dbpedia.org/resource/Ryanair_Holdings",
    "http://dbpedia.org/resource/GAIL_India",
    "http://dbpedia.org/resource/Eastman_Chemical",
    "http://dbpedia.org/resource/CH_Robinson_Worldwide",
    "http://dbpedia.org/resource/Land_Securities_Group",
    "http://dbpedia.org/resource/Latam_Airlines",
    "http://dbpedia.org/resource/Delta_Lloyd",
    "http://dbpedia.org/resource/UC_Rusal",
    "http://dbpedia.org/resource/Banca_MPS",
    "http://dbpedia.org/resource/Alleghany",
    "http://dbpedia.org/resource/Hyundai_Engineering",
    "http://dbpedia.org/resource/Shanghai_International_Port",
    "http://dbpedia.org/resource/St_Jude_Medical",
    "http://dbpedia.org/resource/China_Cosco_Holdings",
    "http://dbpedia.org/resource/Lorillard",
    "http://dbpedia.org/resource/Gjensidige_Forsikring",
    "http://dbpedia.org/resource/LeGrand",
    "http://dbpedia.org/resource/Shimao_Property_Holdings",
    "http://dbpedia.org/resource/Belle_International_Holdings",
    "http://dbpedia.org/resource/Flextronics_International",
    "http://dbpedia.org/resource/WR_Berkley",
    "http://dbpedia.org/resource/Pohjola_Bank",
    "http://dbpedia.org/resource/Hormel_Foods",
    "http://dbpedia.org/resource/T_Rowe_Price",
    "http://dbpedia.org/resource/Zimmer_Holdings",
    "http://dbpedia.org/resource/Brambles",
    "http://dbpedia.org/resource/Delek_Group",
    "http://dbpedia.org/resource/Colruyt",
    "http://dbpedia.org/resource/Chugoku_Electric_Power",
    "http://dbpedia.org/resource/Shoprite_Holdings",
    "http://dbpedia.org/resource/OHL_Group",
    "http://dbpedia.org/resource/Metalurgica_Gerdau",
    "http://dbpedia.org/resource/Hebei_Iron_&_Steel",
    "http://dbpedia.org/resource/Regeneron_Pharmaceuticals",
    "http://dbpedia.org/resource/Greentown_China_Holdings",
    "http://dbpedia.org/resource/GS_Holdings",
    "http://dbpedia.org/resource/Toppan_Printing",
    "http://dbpedia.org/resource/Metro_Inc",
    "http://dbpedia.org/resource/Mitsui_OSK_Lines",
    "http://dbpedia.org/resource/Qingdao_Haier",
    "http://dbpedia.org/resource/Orascom_Construction",
    "http://dbpedia.org/resource/Molson_Coors_Brewing",
    "http://dbpedia.org/resource/Synovus_Financial",
    "http://dbpedia.org/resource/Lennar",
    "http://dbpedia.org/resource/Qantas_Airways",
    "http://dbpedia.org/resource/TUI",
    "http://dbpedia.org/resource/DR_Horton",
    "http://dbpedia.org/resource/Family_Dollar_Stores",
    "http://dbpedia.org/resource/Mead_Johnson_Nutrition",
    "http://dbpedia.org/resource/Coca-Cola_HBC",
    "http://dbpedia.org/resource/Manila_Electric",
    "http://dbpedia.org/resource/Israel_Corp",
    "http://dbpedia.org/resource/Crown_Castle_International",
    "http://dbpedia.org/resource/China_Longyuan_Power",
    "http://dbpedia.org/resource/Zions_Bancorp",
    "http://dbpedia.org/resource/Wuhan_Iron_&_Steel",
    "http://dbpedia.org/resource/Eurasian_Natural_Resources",
    "http://dbpedia.org/resource/DSME",
    "http://dbpedia.org/resource/Supervalu",
    "http://dbpedia.org/resource/JGC",
    "http://dbpedia.org/resource/China_Taiping_Insurance",
    "http://dbpedia.org/resource/Scana",
    "http://dbpedia.org/resource/Rock-Tenn",
    "http://dbpedia.org/resource/Oriental_Land",
    "http://dbpedia.org/resource/Exxaro_Resources",
    "http://dbpedia.org/resource/Angang_Steel",
    "http://dbpedia.org/resource/Banco_BPI",
    "http://dbpedia.org/resource/Hengan_International_Group",
    "http://dbpedia.org/resource/SNC-Lavalin_Group",
    "http://dbpedia.org/resource/Hysan_Development",
    "http://dbpedia.org/resource/American_Water_Works",
    "http://dbpedia.org/resource/Yamaha_Motor",
    "http://dbpedia.org/resource/Petsmart",
    "http://dbpedia.org/resource/Nomos_Bank",
    "http://dbpedia.org/resource/Jindal_Steel_&_Power",
    "http://dbpedia.org/resource/BR_Malls",
    "http://dbpedia.org/resource/CR_Bard",
    "http://dbpedia.org/resource/Wistron",
    "http://dbpedia.org/resource/Kansas_City_Southern",
    "http://dbpedia.org/resource/Pinnacle_West",
    "http://dbpedia.org/resource/Enagas",
    "http://dbpedia.org/resource/Marfrig_Group",
    "http://dbpedia.org/resource/Plains_Exploration",
    "http://dbpedia.org/resource/77_Bank",
    "http://dbpedia.org/resource/Empresas_CMPC",
    "http://dbpedia.org/resource/Kawasaki_Kisen_Kaisha",
    "http://dbpedia.org/resource/Maanshan_Iron_&_Steel",
    "http://dbpedia.org/resource/Quinenco",
    "http://dbpedia.org/resource/TCL_Corp",
    "http://dbpedia.org/resource/NHPC",
    "http://dbpedia.org/resource/Netease",
    "http://dbpedia.org/resource/Serco_Group",
    "http://dbpedia.org/resource/Tiffany_&_Co",
    "http://dbpedia.org/resource/Bank_Muscat",
    "http://dbpedia.org/resource/Mashreq_Bank",
    "http://dbpedia.org/resource/Shanghai_Industrial",
    "http://dbpedia.org/resource/JC_Penney",
    "http://dbpedia.org/resource/Eutelsat_Communications",
    "http://dbpedia.org/resource/DIA",
    "http://dbpedia.org/resource/Doosan_Heavy_Industries",
    "http://dbpedia.org/resource/OGE_Energy",
    "http://dbpedia.org/resource/China_Agri-Industries",
    "http://dbpedia.org/resource/First_Niagara_Financial",
    "http://dbpedia.org/resource/Vesuvius",
    "http://dbpedia.org/resource/Toyo_Seikan_Kaisha",
    "http://dbpedia.org/resource/Telecom_of_New_Zealand",
    "http://dbpedia.org/resource/Chicago_Bridge_&_Iron",
    "http://dbpedia.org/resource/Total_Access_Communication",
    "http://dbpedia.org/resource/Laurentian_Bank",
    "http://dbpedia.org/resource/MISC",
    "http://dbpedia.org/resource/Kuala_Lumpur_Kepong",
    "http://dbpedia.org/resource/Mirvac_Group",
    "http://dbpedia.org/resource/Transurban_Group",
    "http://dbpedia.org/resource/Banque_Centrale_Populaire",
    "http://dbpedia.org/resource/Lindt_&_Sprungli",
    "http://dbpedia.org/resource/Symetra_Financial",
    "http://dbpedia.org/resource/Nippon_Meat_Packers",
    "http://dbpedia.org/resource/Whiting_Petroleum",
    "http://dbpedia.org/resource/Finning_International",
    "http://dbpedia.org/resource/Hakuhodo_DY_Holdings",
    "http://dbpedia.org/resource/Aeroflot-Russian_Airlines",
    "http://dbpedia.org/resource/First_Horizon_National",
    "http://dbpedia.org/resource/Wing_Hang_Bank",
    "http://dbpedia.org/resource/Tractor_Supply",
    "http://dbpedia.org/resource/UOL_Group"
]

# Filter out absent entities
filtered_entities = [e for e in entities if e not in absent_entities]
filtered_labels = [label for entity, label in zip(entities, labels) if entity not in absent_entities]
filtered_reg_targets = [reg_targets for entity, reg_targets in zip(entities, reg_targets) if entity not in absent_entities]

Prepare for Knowledge Graph

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
import networkx as nx

In [ ]:
# Load the graph from the input file
graph = Graph()
graph.parse("/content/drive/My Drive/dbpedia_files/cleaned_Forbes_filtered_dbpedia.ttl", format="turtle")

<Graph identifier=N35826d3397fa47dea2f5fc0d56f779b8 (<class 'rdflib.graph.Graph'>)>

In [ ]:
G = nx.DiGraph()  # Create a directed graph

for s, p, o in graph:
    G.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G.nodes())
num_edges = len(G.edges())
print("The DBpedia knowledge graph as of 2016-10 has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The DBpedia knowledge graph as of 2016-10 has 3976256 nodes and 11546003 edges.


Classification & Regression tasks based graph embeddings without SA

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error
import pandas as pd

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('pure_class_reg_results.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.563802
1           cbow         50                 C4.5  Accuracy   0.517994
2           cbow         50            KNN (k=3)  Accuracy   0.546582
3           cbow         50                  SVM  Accuracy   0.524772
4           cbow         50    Linear Regression      RMSE  31.780191


Classification & Regression tasks based graph embeddings extracted on subgraph base on SA with BFS

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving main.py to main.py
Saving sa_helper.py to sa_helper.py


In [ ]:
from main import *
from sa_helper import *

In [ ]:
# Count the neighbor node number for each nodes in the graph
neighbor_counts = {node: len(list(G.neighbors(node))) for node in G.nodes()}

excl=False, pop=False

In [ ]:
subgraph_F_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub.nodes())
num_edges = len(G_sub.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 33956 nodes and 46633 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_ttl_class_reg_results_ff.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.525592
1           cbow         50                 C4.5  Accuracy   0.568461
2           cbow         50            KNN (k=3)  Accuracy   0.627028
3           cbow         50                  SVM  Accuracy   0.599949
4           cbow         50    Linear Regression      RMSE  29.178628


excl=True, pop=False

In [ ]:
subgraph_T_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=True, pop=False)

In [ ]:
G_sub_tf = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_F:
    G_sub_tf.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tf.nodes())
num_edges = len(G_sub_tf.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 33956 nodes and 46633 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_tf.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_tf.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_tf.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.518051
1           cbow         50                 C4.5  Accuracy   0.563927
2           cbow         50            KNN (k=3)  Accuracy   0.614998
3           cbow         50                  SVM  Accuracy   0.613465
4           cbow         50    Linear Regression      RMSE  29.874605


excl=False, pop=True

In [ ]:
subgraph_F_T = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts, alpha=0.186,
                                        beta=0.006, max_hops=3, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=True)

In [ ]:
G_sub_ft = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_T:
    G_sub_ft.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ft.nodes())
num_edges = len(G_sub_ft.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 33956 nodes and 46633 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_T.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ft.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ft.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_ft.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.520335
1           cbow         50                 C4.5  Accuracy   0.556380
2           cbow         50            KNN (k=3)  Accuracy   0.608943
3           cbow         50                  SVM  Accuracy   0.615043
4           cbow         50    Linear Regression      RMSE  28.618589


excl=True, pop=True

In [ ]:
subgraph_T_T = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts, alpha=0.186,
                                        beta=0.006, max_hops=3, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=True, pop=True)

In [ ]:
G_sub_tt = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_T:
    G_sub_tt.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tt.nodes())
num_edges = len(G_sub_tt.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 33956 nodes and 46633 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_T.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_tt.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_tt.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_class_reg_results_tt.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.524066
1           cbow         50                 C4.5  Accuracy   0.587241
2           cbow         50            KNN (k=3)  Accuracy   0.609700
3           cbow         50                  SVM  Accuracy   0.591718
4           cbow         50    Linear Regression      RMSE  28.810306


Classification & Regression tasks based graph embeddings extracted on subgraph base on SA with DFS

excl=False, pop=False

In [ ]:
subgraph_F_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub_ff = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub_ff.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ff.nodes())
num_edges = len(G_sub_ff.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 33955 nodes and 46633 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_ff.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.514291
1           cbow         50                 C4.5  Accuracy   0.554802
2           cbow         50            KNN (k=3)  Accuracy   0.615755
3           cbow         50                  SVM  Accuracy   0.595443
4           cbow         50    Linear Regression      RMSE  28.941277


excl=True, pop=False

In [ ]:
subgraph_T_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.1, beta=0.009, max_depth=2, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=True, pop=False)

In [ ]:
G_sub_tf = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_F:
    G_sub_tf.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tf.nodes())
num_edges = len(G_sub_tf.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 33955 nodes and 46633 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_tf.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_tf.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_tf.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.514303
1           cbow         50                 C4.5  Accuracy   0.566912
2           cbow         50            KNN (k=3)  Accuracy   0.606704
3           cbow         50                  SVM  Accuracy   0.598445
4           cbow         50    Linear Regression      RMSE  28.757347


excl=False, pop=True

In [ ]:
subgraph_F_T = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=True)

In [ ]:
G_sub_ft = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_T:
    G_sub_ft.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ft.nodes())
num_edges = len(G_sub_ft.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 33955 nodes and 46633 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_T.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ft.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ft.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_ft.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.519560
1           cbow         50                 C4.5  Accuracy   0.558573
2           cbow         50            KNN (k=3)  Accuracy   0.614257
3           cbow         50                  SVM  Accuracy   0.600706
4           cbow         50    Linear Regression      RMSE  28.840317


excl=True, pop=True

In [ ]:
subgraph_T_T = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=True, pop=True)

In [ ]:
G_sub_tt = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_T:
    G_sub_tt.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tt.nodes())
num_edges = len(G_sub_tt.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 33955 nodes and 46633 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_T.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_tt.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_tt.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_class_reg_results_tt.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.521839
1           cbow         50                 C4.5  Accuracy   0.573644
2           cbow         50            KNN (k=3)  Accuracy   0.622551
3           cbow         50                  SVM  Accuracy   0.595466
4           cbow         50    Linear Regression      RMSE  29.287309
